In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras import backend as K
from keras.utils.np_utils import to_categorical

import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
from sklearn.utils import class_weight
from sklearn.metrics import confusion_matrix

from PIL import Image

import time

Using TensorFlow backend.


In [2]:
x_train = []
#x_test = []
y_train = []

df_train = pd.read_csv('train_v2.csv')
#df_test = pd.read_csv('sample_submission_v2.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values[:40479], miniters=1000):
    #img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    y_train.append(targets)

y_train = np.array(y_train, np.uint8)

print(y_train.shape)

100%|██████████| 40479/40479 [00:00<00:00, 271628.67it/s]

(40479, 17)


In [3]:
for i in range(17):
    if(np.sum(y_train[:,i],axis=0)>y_train.size/2):
        l = np.where(y_train[:,i] == 0)[0]
        for j in range(1000):
            ind = np.random.choice(l)
            filename = df_train.values[ind,0]
            img = Image.open("train-jpg/"+filename+".jpg")
            rnd = np.random.choice([1,2,3,4])
            if rnd == 1:
                img2 = img.transpose(Image.FLIP_LEFT_RIGHT)
            elif rnd == 2:
                img2 = img.transpose(Image.FLIP_TOP_BOTTOM)
            elif rnd == 3:
                img2 = img.transpose(Image.ROTATE_180)
            elif rnd == 4:
                img2 = img.transpose(Image.TRANSPOSE)
            img2.save("train-augmented/"+filename+"_"+str(i)+"_"+str(j)+".jpg")
    else:
        l = np.where(y_train[:,i] == 1)[0]
        for j in range(1000):
            ind = np.random.choice(l)
            filename = df_train.values[ind,0]
            img = Image.open("train-jpg/"+filename+".jpg")
            rnd = np.random.choice([1,2,3,4])
            if rnd == 1:
                img2 = img.transpose(Image.FLIP_LEFT_RIGHT)
            elif rnd == 2:
                img2 = img.transpose(Image.FLIP_TOP_BOTTOM)
            elif rnd == 3:
                img2 = img.transpose(Image.ROTATE_180)
            elif rnd == 4:
                img2 = img.transpose(Image.TRANSPOSE)
            img2.save("train-augmented/"+filename+"_"+str(i)+"_"+str(j)+".jpg")                      

['train_10203_11_318.jpg', 'train_10203_11_207.jpg']

selective_logging
Train on 36431 samples, validate on 4048 samples
Epoch 1/5
14s - loss: 0.2798 - acc: 0.9907 - val_loss: 0.0956 - val_acc: 0.9941
Epoch 2/5
14s - loss: 0.2796 - acc: 0.9913 - val_loss: 0.0956 - val_acc: 0.9941
Epoch 3/5
14s - loss: 0.2796 - acc: 0.9913 - val_loss: 0.0956 - val_acc: 0.9941
Epoch 4/5
14s - loss: 0.2796 - acc: 0.9913 - val_loss: 0.0956 - val_acc: 0.9941
Epoch 5/5
14s - loss: 0.2796 - acc: 0.9913 - val_loss: 0.0956 - val_acc: 0.9941
[[4024    0]
 [  24    0]]


array([ 1895,   316,   295,  3326,    87, 33746,   309, 25614,   190,
        4025,  6490,   780,    87,  2431, 11061,  7276,  6652], dtype=uint64)

'road'

100%|██████████| 61191/61191 [01:34<00:00, 645.74it/s]


TypeError: only integer scalar arrays can be converted to a scalar index

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,primary partly_cloudy clear
4,test_4,agriculture cloudy primary partly_cloudy
5,test_5,primary clear
6,test_6,agriculture primary habitation cultivation wat...
7,test_7,road agriculture habitation clear
8,test_8,primary clear
9,test_9,primary partly_cloudy
